In [ ]:
!pip install git+https://github.com/a-r-r-o-w/stablefused ipython

In [5]:
import numpy as np
import torch

from IPython.display import Video, display
from stablefused import TextToImageDiffusion, TextToImageConfig
from stablefused.utils import image_grid, pil_to_video

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
# model_id = "segmind/tiny-sd"
model = TextToImageDiffusion(
    model_id=model_id, revision="fp16", torch_dtype=torch.float16
)

In [3]:
model.set_scheduler("dpm_multistep")

In [4]:
prompt = "Cyberpunk cityscape with towering skyscrapers, neon signs, and flying cars."
negative_prompt = "cartoon, unrealistic, blur, boring background, deformed, disfigured, low resolution, unattractive"
num_inference_steps = 20
seed = 1337

torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
config = TextToImageConfig(prompt=prompt, num_inference_steps=num_inference_steps)
model(config)[0]

In [7]:
prompt = [
    "Gothic painting of an ancient castle at night, with a full moon, gargoyles, and shadows",
    "A lion in galaxies, spirals, nebulae, stars, smoke, iridescent, intricate detail, octane render, 8k",
    "A close up image of an very beautiful woman, aesthetic, realistic, high quality",
    "Concept art for a post-apocalyptic world with ruins, overgrown vegetation, and a lone survivor",
]

In [8]:
model.enable_attention_slicing()
model.enable_slicing()
model.enable_tiling()

In [ ]:
config = TextToImageConfig(
    prompt=prompt,
    negative_prompt=[negative_prompt] * len(prompt),
    num_inference_steps=20,
    guidance_scale=10.0,
    return_latent_history=True,
)
images = model(config)

In [10]:
timestep_images = []
for imgs in zip(*images):
    img = image_grid(imgs, rows=2, cols=len(prompt) // 2)
    timestep_images.append(img)

In [11]:
path = "text_to_image_diffusion.mp4"
pil_to_video(timestep_images, path, fps=5)

In [ ]:
display(Video(path, embed=True))